In [1]:
import nibabel as nib
import numpy as np
from PIL import Image, ImageOps

In [ ]:
# Path to the original NIfTI image (used only for shape and affine reference)
nifti_path = r"test\neurocranium_ct_image.nii"
nifti_img = nib.load(nifti_path)
nifti_data = nifti_img.get_fdata()
reference_shape = nifti_img.shape      # Dimensions of the reference NIfTI
affine = nifti_img.affine              # Spatial orientation matrix

# Path to the folder containing the sagittal-plane segmentation masks (PNG format)
png_base_path = "test\segmentation_masks_folder_sagittal_plane\diploe_mask"

# Path where the output NIfTI file will be saved
output_nifti_path = "test\diploe_mask.nii"

# List of PNG slice indices to be used, and their corresponding NIfTI slice indices
# Note: In this example, sagittal slices are indexed from the opposite end,
# so the mapping is reversed using (511 - idx).
png_slices = range(126, 383)
nifti_slices = [511 - idx for idx in png_slices]

# Initialize an empty 3D volume (same shape as the reference NIfTI)
volume_3d = np.zeros(reference_shape)

# Loop over PNG slices and insert them into the correct NIfTI slice position
for png_idx, nifti_idx in zip(png_slices, nifti_slices):
    png_path = f"{png_base_path}/Slice_{png_idx}_diploe_mask.png"
    try:
        # Load the PNG mask as grayscale
        img = Image.open(png_path).convert('L')

        # Flip vertically to match NIfTI orientation
        img_flipped = ImageOps.flip(img)

        # Resize to match NIfTI in-plane dimensions (Y, Z axes)
        img_resized = img_flipped.resize(
            (reference_shape[1], reference_shape[2]),
            Image.Resampling.LANCZOS
        )

        # Convert to NumPy array
        png_array = np.array(img_resized)

        # Insert into the 3D volume (transpose to correct orientation)
        volume_3d[nifti_idx, :, :] = png_array.T

    except FileNotFoundError:
        # If the PNG is missing, skip it and print a warning
        print(f"⚠️ File not found: {png_path}. Slice skipped.")
        continue

# Create and save the new NIfTI file
new_nifti = nib.Nifti1Image(volume_3d, affine)
nib.save(new_nifti, output_nifti_path)

print(f"✅ NIfTI file saved at: {output_nifti_path}")

✅ File NIfTI salvato in: test\diploe_mask.nii
